In [ ]:
import re
import codecs
import random
class relations:
    def __init__(self):
        self.tag_dic = {"has_clinical_manifestation":"临床表现",
           "cause":"病因",
           "diagnose":"诊断",
           "medical_treat":"治疗",
           "prevent":"预防",
           "is_synonym":"同义",
           'is_synonym_drug':"同义",
           'is_synonym_mechanism':"同义",
           'is_synonym_cause':"同义",
           'is_synonym_surgery': "同义",
           'is_synonym_symptom':"同义",
           'is_synonym_prevention':"同义",
           'is_synonym_inspection':"同义",
           'is_synonym_technology':"同义",
           'is_synonym_therapy':"同义",
           'is_synonym_nursing':"同义",
           'is_synonym_indicator':"同义",
           "is_pathology":"病理",
           "nurse":"护理",
           "report":"报告",
           "inspect":"观察",
           "non-medication_treat":"治疗",
           "surgical_treat":"治疗",
           "disease": "疾病",
           "drug": "药物",
           "surgery": "手术",
           "symptom": "症状",
           "prevention": "预防措施",
           "inspection_method": "观察方法",
           "diagnostic_technology": "诊断技术",
           "non-drug_therapy": "非药疗法",
           "nursing_technology": "护理技术",
           "cause_of_disease": "病因",
           "pathological_mechanism": "病理",
           'medical_indicator': '医疗指标',
           'has_subclass':'分为',
           'has_indicator':'检测项目为',
           'has_inspector':'检测项目为',
           'drug_effect':'作用于',
           'drug_effect_symptom':'作用于',
           'drug_effect_pathological':'作用于',
           'non-drug_therapy_effect':'作用于',
           'prevention_effect':'作用于',
           'drug_cause_of_disease':'作用于',
           'lead':"导致",
           'diagnostic_pathological':'诊断',
           'indicate_symptom':"揭示",
           'nursing_symptom':'护理',
            'is_complication':'并发症'
           
          }
    def get_relation(self,ann_file):
        with codecs.open(ann_file, "r", encoding="utf-8") as f:#读取标签文件
            line = f.readline()#获取第一行
            print(line)
            line = line.strip("\n\r")
            entity=[]#标注文件中的实体储存在此处
            relation=[]#标注文件中的关系储存在此处
            while line != "":
                line_arr = line.split()
                cls = self.tag_dic[line_arr[1]] #转换为中文标签
                line_arr[1]=cls#中文标签替换
                if "T" in line_arr[0]:
                    entity.append(line_arr)#实体存入entity
                else:
                    relation.append(line_arr)#关系存入relation
                print(line_arr)
                line = f.readline()#读取下一个
                line = line.strip("\n\r")
            for i in range(len(relation)):
                for j in range(len(entity)):
                    if entity[j][0]==relation[i][2][5:]:
                        relation[i][2]=entity[j][2:]
                    if entity[j][0]==relation[i][3][5:]:
                        relation[i][3]=entity[j][2:]
        return relation,entity
    def sentence_trans(self,txt):
        with codecs.open(txt, "r", encoding="utf-8") as f:#读取文本文档
            content_str = f.read()
            con=content_str.replace("\r\n","。。")
        return con
    def clean_sen(self,con):
        p1 = r"。+"
        pattern1 = re.compile(p1)
        print(pattern1.findall(con))
        newKey = re.sub(p1, "。", con)
        sen_full=newKey.split("。")
        cleaned_sen=[]
        for i in sen_full:
            i=i.replace("(",'')
            i=i.replace(")",'')
            i=i.replace(" ",'')
            i=i.replace("Alzheimer's","Alzheimer'sdisease")
            cleaned_sen.append(i)
        return cleaned_sen
    def mark_pos(self,con):
        full_mark_list=[]
        for i,full_mark in enumerate(con):
            if (full_mark=="。") :
                full_mark_list.append(i)
        del_li=[]
        for i in range(len(full_mark_list)):
            try:
                if full_mark_list[i+1]-full_mark_list[i]==1:
                    del_li.append(full_mark_list[i+1])
            except:
                pass
        cleaned_li=[]
        for i in full_mark_list:
            if i not in del_li:
                cleaned_li.append(i)
        return cleaned_li
    def get_triple(self,con,relation):
        cleaned_li=self.mark_pos(con)
        cleaned_sen=self.clean_sen(con)
        relation_sen=[]
        for i in relation:
            i[2][2]=i[2][2].replace("(",'')
            i[2][2]=i[2][2].replace("(",'')
            i[2][2]=i[2][2].replace(" ",'')
            i[2][2]=i[2][2].replace("Alzheimer's","Alzheimer'sdisease")
            i[3][2]=i[3][2].replace(")",'')
            i[3][2]=i[3][2].replace(")",'')
            i[3][2]=i[3][2].replace(" ",'')
            i[3][2]=i[3][2].replace("Alzheimer's","Alzheimer'sdisease")
            try:
                for j in range(len(cleaned_li)):
                        if int(i[2][0])>int(cleaned_li[j]) and int(i[2][0])<int(cleaned_li[j+1]) and int(i[3][0])>int(cleaned_li[j]) and int(i[3][0])<int(cleaned_li[j+1]):
                            relation_sen.append([i[1],i[2][2],i[3][2],j,cleaned_sen[j+1]])#中间的句子也要放进去 我需要句子的index
            except:
                pass
        return relation_sen
    def run(self):
        relation,entity=self.get_relation(r".\关系抽取\abstract_000.ann")
        con=self.sentence_trans(r".\关系抽取\abstract_000.txt")
        
        
        relation_sen=self.get_triple(con,cleaned_li,cleaned_sen,relation)
        sample=self.neg_sample(cleaned_sen,cleaned_li,relation_sen,entity,relation,500)
        w_path="./test.txt"
        with codecs.open(w_path, "w", encoding="utf-8") as w:
            for i in sample:
                w.write(i[0]+"\t"+i[1]+"\t"+"unknown"+"\t"+i[2])
                w.write("\r\n")
    def neg_sample(self,cleaned_sen,cleaned_li,relation_sen,entity,relation,sample_size):
    #制作负样本 找出句子中所有的实体
        sen_for_unkn=[]
        for i in range(len(cleaned_sen)):
            sen_for_unkn.append([cleaned_sen[i]])
        for i in entity:
            for j in range(len(cleaned_li)):
                if int(i[2])>int(cleaned_li[j]) and int(i[2])<int(cleaned_li[j+1]):
                    sen_for_unkn[j+1].append(i[4])
        relation_pair=[]
        for i in relation_sen:
            if [i[1],i[2]] not in relation_pair:
                relation_pair.append([i[1],i[2]])
        syn_word=[]
        for i in relation:
            if i[1]=="同义":
                syn_word.append([i[2][2],i[3][2]])
        relat_plus_syn=[]
        for i in relation_pair:
            for j in syn_word:
                if i[0] ==j[0]:
                    relat_plus_syn.append([j[1],i[1]])
                if i[1] ==j[0]:
                    relat_plus_syn.append([i[0],j[1]])
                if i[0] ==j[1]:
                    relat_plus_syn.append([j[0],i[1]])
                if i[1] ==j[1]:
                    relat_plus_syn.append([i[0],j[0]])
        plused_reli=relation_pair+relat_plus_syn
        unkn_li=[]
        for i in sen_for_unkn:
            for j in i[1:]:
                for k in i[1:]:
                    if j!=k and(([j,k]  not in plused_reli) and ([k,j] not in plused_reli)) and (j!="老年痴呆" or j!='AD') and (k!="老年痴呆" or k!='AD'):
                        unkn_li.append([j,k,i[0]])
        sample=random.sample(unkn_li, sample_size)
        return sample